# Session 5: Transformer-based NMT

Investigating the Transformer Model
===================================

In `/opt/data/nmt_models/transformer/` you can find a Transformer model
file, as well as the corresponding training log file which stores the
output from the training of the Transformer model. As you can see from
the model description at the beginning of the log file, the Transformer
is much more complicated than the LSTM-based NMT model that we saw
before.

The following questions aim to help you better understand the structure
of this model. As the title of the original paper describing this model
architecture *"Attention is all you need"* suggests, this model only
consists of attention layers and fully connected layers (basically MLPs
with ReLU as the activation function).

1)  Find and pick one of the many `MultiHeadAttention` modules in the
    model description. How many parameters are there in each
    `MultiHeadAttention` module?

```
(multihead): MultiHeadAttention(
  (fc_query): Bottle(
    (function): XavierLinear(in_features=512, out_features=512, bias=False, weight_norm=False)
  )
  (fc_key): Bottle(
    (function): XavierLinear(in_features=512, out_features=512, bias=False, weight_norm=False)
  )
  (fc_value): Bottle(
    (function): XavierLinear(in_features=512, out_features=512, bias=False, weight_norm=False)
  )
  (fc_concat): Bottle(
    (function): XavierLinear(in_features=512, out_features=512, bias=False, weight_norm=False)
  )
  (attn_dropout): Dropout(p=0.2)
)
```

- 4 `Linear` layers a $512 \times 512$ features  
  $\implies 4 \times M \in \mathbb{R}^{512 \times 512}$  
  $\implies 4 \cdot (2^9)^2 = 2^{20} = 1.05\text{M}$ parameters

2)  Given the sequence length $T=50$, batch size $B=200$ (number of
    sentences) and hidden size $H=512$. Can you estimate the memory
    required for each self-attention layer of the network?  
    *Hint:* the layer has to store $4$ outputs of size $B * T * H$
    neurons. Each neuron is represented by a $4$ byte floating-point
    number. Additionally, we also use the same amount of memory for the
    gradients.

$4 \cdot T \cdot B \cdot H = 40000 \cdot 512 = 20480000$ floats activation  
$= 81920000$ bytes for the activation  
$= 78 \text{MB}$  

156 MB for activation + gradient per self-attention layer

3)  Can you give an estimate of how many parameters there are in the
    whole model (the answer is at the beginning of the file)? More
    importantly, what is the ratio of parameters between Embedding,
    Output layer and Hidden layers.  
    *Note:* this model uses parameter sharing across Encoder embeddings,
    Decoder embeddings and the Decoder Output layer. As a result we just
    have a single embedding matrix in the whole model, which is a trick
    to significantly reduce the total number of parameters.

- $1 \times$ word embedding
  - embedding size 512
  - 40104 embedding vectors  
  $\implies 20.5\text{M}$ parameters in embedding/softmax layer

- $8 \times$ Encoder layer, $8 \times$ Decoder layer
  - 16 self-attention layers, 16 feed-forward layers  
  $\implies 16 \cdot 3.15\text{M} = 50.4\text{M}$ parameters (see (4))  
  - some additional parameters in the Decoder for Encoder-Decoder attention

| Module     |Parameters|
|------------|----------|
| encoder    |   31.54% |
| decoder    |   42.05% |
| embeddings |   25.70% |

4)  One layer in the Encoder consists of one "MultiHeadAttention" and
    one "FeedForward". Can you compare the number of parameters of such
    layer compared to one LSTM layer of the same size?  
    *Hint:* an LSTM with 512 hidden units has around 2.1M parameters

- MultiHeadAttention: $2^{20} = 1.05\text{M}$ parameters  
- FeedForward: $512 \cdot 2048 + 2048 \cdot 512 = 2 \cdot (2^{11} \cdot 2^9) = 2^{21} = 2.1\text{M}$ parameters  
- Total: 3.15M parameters
- 50% more parameters than the LSTM layer

Multilingual Transformer Model
==============================

The above model is a multilingual translation model. It translates
between English and 4 other European languages (German, Dutch, Italian,
Romanian).

We have prepared a multilingual translation dataset in
`/opt/data/iwslt2017/`.  
In order to load the Transformer model and use it for translation, you
should also clone the following repository using  
`$ git clone --branch DbMajor https://github.com/quanpn90/NMTGMinor.git`

In [1]:
git clone --branch DbMajor https://github.com/quanpn90/NMTGMinor.git

Cloning into 'NMTGMinor'...
remote: Enumerating objects: 6691, done.
remote: Counting objects: 100% (1327/1327), done.
remote: Compressing objects: 100% (812/812), done.
remote: Total 6691 (delta 1015), reused 780 (delta 498), pack-reused 5364
Receiving objects: 100% (6691/6691), 2.80 MiB | 10.41 MiB/s, done.
Resolving deltas: 100% (5039/5039), done.


In the `iwslt2017` directory you can find test datasets for different
language pairs. The filename format of these datasets is
`test.${src}-${tgt}.bpe.${src}` and
`test.${src}-${tgt}.bpe.${tgt}`, where `${src}` indicates the
source language and `${tgt}` indicates the target language.

In [2]:
ls /opt/data/iwslt2017/ | grep test.en-..

test.en-de.bpe.de
test.en-de.bpe.en
test.en-it.bpe.en
test.en-it.bpe.it
test.en-nl.bpe.en
test.en-nl.bpe.nl
test.en-ro.bpe.en
test.en-ro.bpe.ro


We also provide a trained model which you can find at
`/opt/data/nmt_models/transformer/model.average.pt`.

To then translate from a testset use

```bash
$ python3 NMTGMinor/translate.py \
    -gpu 0 \
    -model <your-model> \
    -src <your-input-file> \
    -output <your-output-file> \
    -beam_size 1 \
    -batch_size 256 \
    -bos_token <language-tag> \
    -start_with_tag \
    -verbose
```

After decoding we want to calculate the translation BLEU score, for
which we use the human reference translations in
`/opt/data/iwslt2017/nobpe/`:

`$ sed s/@@ //g | multi-bleu.perl <reference-file> `  
This command first removes the BPE annotations and then calls the script
`multi-bleu.perl` script to calculate the BLEU score.

1)  Translate the English testsets (the test files that have English as
    their source language) to their respective target language indicated
    in the test data filename and report the BLEU scores.

In [3]:
for tgtlang in de nl it ro
do
    echo "Translating en->${tgtlang}..."
    srcfile=/opt/data/iwslt2017/test.en-$tgtlang.bpe.en
    outfile=/tmp/test.en-$tgtlang.bpe.hyp.$tgtlang
    
    python3 NMTGMinor/translate.py \
        -gpu 0 \
        -model /opt/data/nmt_models/transformer/model.average.pt \
        -src $srcfile \
        -output $outfile \
        -beam_size 1 \
        -batch_size 256 \
        -bos_token "#${tgtlang}#" \
        -start_with_tag
done

Translating en->de...
* Starting token #de# 
['/opt/data/nmt_models/transformer/model.average.pt']
* Joining the weights of encoder and decoder word embeddings
* Joining the weights of decoder input and output embeddings
Translating en->nl...
* Starting token #nl# 
['/opt/data/nmt_models/transformer/model.average.pt']
* Joining the weights of encoder and decoder word embeddings
* Joining the weights of decoder input and output embeddings
Translating en->it...
* Starting token #it# 
['/opt/data/nmt_models/transformer/model.average.pt']
* Joining the weights of encoder and decoder word embeddings
* Joining the weights of decoder input and output embeddings
Translating en->ro...
* Starting token #ro# 
['/opt/data/nmt_models/transformer/model.average.pt']
* Joining the weights of encoder and decoder word embeddings
* Joining the weights of decoder input and output embeddings


In [4]:
for tgtlang in de nl it ro
do
    echo "en->${tgtlang}"
    outfile=/tmp/test.en-$tgtlang.bpe.hyp.$tgtlang
    reffile=/opt/data/iwslt2017/nobpe/test.en-$tgtlang.$tgtlang
    
    sed 's/@@ //g' $outfile |
        multi-bleu.perl $reffile 2> /dev/null
done

en->de
BLEU = 30.84, 63.1/36.7/24.0/16.3 (BP=1.000, ratio=1.010, hyp_len=31047, ref_len=30744)
en->nl
BLEU = 34.58, 67.6/41.5/28.1/19.6 (BP=0.982, ratio=0.982, hyp_len=29802, ref_len=30357)
en->it
BLEU = 32.30, 62.0/38.4/25.8/17.7 (BP=1.000, ratio=1.008, hyp_len=29203, ref_len=28970)
en->ro
BLEU = 30.18, 61.0/36.7/24.0/16.0 (BP=0.991, ratio=0.991, hyp_len=30394, ref_len=30678)


2)  Did you notice that sometimes the model still translates to the
    wrong language? In training the model only sees English paired with
    all of the 4 other languages.  
    Now try to translate from German (de) to Dutch (nl), thus
    translating between two languages the model has never explicitly
    been trained to produce translations for. This is a so called
    *zero-shot* translation task, for which the model has to rely on its
    learned sentence representation to be independent of the source
    language. Try translating in two different ways:

a)  Direct $de{\rightarrow}nl$ translation through indication of the
    target language via `-bos_token #nl#`.

In [5]:
srclang=de
tgtlang=nl

# srcfile=/opt/data/iwslt2017/test.$srclang-$tgtlang.bpe.$srclang
# outfile=/tmp/test.$tgtlang-$tgtlang.bpe.hyp.$tgtlang
srcfile=/opt/data/iwslt2017/tst2017/tst2017.$srclang-$tgtlang.bpe.$srclang
outfile=/tmp/tst2017.$srclang-$tgtlang.bpe.hyp.$tgtlang

python3 NMTGMinor/translate.py \
    -gpu 0 \
    -model /opt/data/nmt_models/transformer/model.average.pt \
    -src $srcfile \
    -output $outfile \
    -beam_size 1 \
    -batch_size 256 \
    -bos_token "#${tgtlang}#" \
    -start_with_tag \
    -verbose

* Starting token #nl# 
['/opt/data/nmt_models/transformer/model.average.pt']
Loading model from /opt/data/nmt_models/transformer/model.average.pt
* Joining the weights of encoder and decoder word embeddings
* Joining the weights of decoder input and output embeddings
Done
SENT 1: #de# das ist James R@@ isen .
PRED 1: dit is James R@@ isen .
PRED SCORE: -1.2667

SENT 2: #de# sie kennen ihn vielleicht , weil er als New - York - Times - Re@@ por@@ ter den Pul@@ itzer - Preis gewann .
PRED 2: je kent hem misschien , want als New York Times - verslag@@ gever won hij de Pul@@ itzer Prize .
PRED SCORE: -9.6380

SENT 3: #de# lange bevor je jemand von Edward Snowden gehört hatte , schrieb R@@ isen ein Buch , in dem er spektakul@@ är veröffent@@ lichte , dass die NSA illegal Telef@@ one von Amerikan@@ ern ab@@ gehört habe .
PRED 3: lang voordat iemand Edward Snowden had gehoord , schreef R@@ isen een boek waarin hij spectacul@@ air public@@ eerde dat de NSA illegale tele@@ phones van Amerikanen 

PRED SCORE: -20.9854

SENT 28: #de# und zwar können sie das von dem Moment an , in dem sie Kontakt aufnehmen , anstatt im Nach@@ hinein , im Ze@@ ugen@@ stand .
PRED 28: en dat is dat ze dat kunnen doen vanaf het moment dat ze contact opnemen , in plaats van in het achter@@ hoofd , in het test@@ je .
PRED SCORE: -18.1543

SENT 29: #de# heute gibt es Kommunikations@@ software , die es noch nicht gab , als R@@ isen sein Buch geschrieben hatte , und es ist wesentlich sicherer als normale E - Mails oder Telefon@@ an@@ ru@@ fe .
PRED 29: er is nu communication software die nog niet bestond toen R@@ isen zijn boek had geschreven , en het is veel safer dan normale e - mails of telefoon@@ gesprekken .
PRED SCORE: -19.2200

SENT 30: #de# eine solche Technologie ist Sec@@ ure@@ Dro@@ p , ein Open - Source - Über@@ mitt@@ lungs@@ system für Wh@@ ist@@ le@@ blo@@ wer , das ursprünglich von der Internet@@ berühm@@ heit , dem von uns ge@@ gan@@ genen Aaron S@@ war@@ tz , entwickelt wurde , und ist h

PRED 49: er is een hoop rijke mensen en anderen die nervous wachten op de volgende public@@ atie , en op de volgende onth@@ ul@@ de document@@ ai@@ res .
PRED SCORE: -20.8166

SENT 50: #de# nun klingt das wie der P@@ lot eines A@@ gent@@ enth@@ r@@ ill@@ ers oder eines John - G@@ ris@@ ham - Rom@@ ans .
PRED 50: dat klinkt als de sp@@ orten van een toe@@ gang@@ sh@@ an@@ ger@@ es of een John - G@@ ris@@ ham - roman@@ s .
PRED SCORE: -23.3896

SENT 51: #de# es scheint wein@@ t entfernt von Ihnen , mir , gewöhn@@ lichen Leuten .
PRED 51: het lijkt een beetje te liggen van jullie , mij , gewone mensen .
PRED SCORE: -13.6219

SENT 52: #de# warum sollte uns das was angehen ?
PRED 52: waarom zouden we dit aanpakken ?
PRED SCORE: -3.8494

SENT 53: #de# doch die Wahrheit ist : wenn die Re@@ ichen und M@@ äch@@ tigen in der Lage sind , ihr Vermögen in Ste@@ u@@ ero@@ asen zu verstecken und nicht alle ihre Ste@@ uern zu bezahlen , dann bedeutet dies weniger Geld für wichtige öffentliche Dien@@ s

PRED 79: het is niet om een enkele advocaat die onze ver@@ stopte mede@@ werker voor@@ doet .
PRED SCORE: -16.5485

SENT 80: #de# es geht nicht um einen einzelnen Spit@@ zen@@ polit@@ iker , verw@@ ickelt in einen Sk@@ and@@ al .
PRED 80: het is niet één enkele top@@ polit@@ icus , die in een scand@@ al zit .
PRED SCORE: -12.5045

SENT 81: #de# es geht darum , wie das System funktioniert , durch das Korruption , Steuer@@ flu@@ cht , Armut und In@@ stabil@@ ität etabli@@ ert werden .
PRED 81: het gaat over hoe het systeem werkt , door het establ@@ eren van corruption , belasting@@ ont@@ ste@@ king , armoede en inst@@ abil@@ iteit .
PRED SCORE: -16.9412

SENT 82: #de# und um dies zu bewältigen , müssen wir die Spiel@@ re@@ geln ändern .
PRED 82: om dit te kunnen doen , moeten we de spel@@ regels veranderen .
PRED SCORE: -8.3259

SENT 83: #de# wir müssen die Regeln ändern , um diese Art von Verhalten zu ersch@@ weren .
PRED 83: we moeten de regels veranderen om dit soort behavior te ont@@

PRED 112: dit is een kleine blo@@ em@@ b@@ ak .
PRED SCORE: -9.0761

SENT 113: #de# dann hörte ich mich sagen : " ich habe eine Luft@@ mat@@ rat@@ ze . du kannst in meinem Wohn@@ zimmer schlafen . "
PRED 113: ik hoorde me zeggen : " ik heb een lucht@@ mat@@ te . je kan in mijn woon@@ kamer slapen . "
PRED SCORE: -11.0008

SENT 114: #de# eine Stimme in meinem Kopf sagte : " ä@@ h , wie bitte ? "
PRED 114: een stem in mijn hoofd zei : " eh , hoe ga je ? "
PRED SCORE: -10.0340

SENT 115: #de# nachts lag ich dann im Bett , ich star@@ r@@ te an die Decke und dachte : " Oh Mann ! was hab ich mir da ein@@ geb@@ roc@@ kt ?
PRED 115: ik lay in bed en ik keek naar de ceiling en dacht : " man ! wat heb ik in de war gezet ?
PRED SCORE: -19.2162

SENT 116: #de# ein Wild@@ fre@@ m@@ der schl@@ äft in meinem Wohn@@ zimmer .
PRED 116: een wil@@ der@@ s@@ moeder sla@@ apt in mijn woon@@ kamer .
PRED SCORE: -8.8509

SENT 117: #de# was , wenn er verrückt ist ? "
PRED 117: wat als hij gek is ? "
PRED SCOR

...

PRED SCORE: -41.1071

SENT 4023: #de# und wenn Du es schaf@@ fst , gebe ich D@@ ir Hunder@@ t Dollar . "
PRED 4023: als je het lukt , geef ik je 100 dollar . "
PRED SCORE: -6.9903

SENT 4024: #de# das war natürlich nur ein einfacher Trick von Dr@@ .@@ P. , damit ich die Ü@@ bungen mache , die ich nicht machen wollte , mit der Aus@@ sicht , dass reich@@ ste fünf@@ jährige Mädchen in der Kranken@@ station im zweiten Stock zu werden . aber er hat mich tatsächlich dazu gebracht , meinen täglichen Ü@@ b@@ ungs - Hor@@ ror mit neuen Augen zu sehen . so wurde daraus ein neues und viel@@ verspre@@ chen@@ des Erlebnis .
PRED 4024: dat was natuurlijk een simpele truc van Dr@@ .@@ P. , om de oef@@ eningen te doen die ik niet wanted te doen , met de besef dat de rich@@ igste vijf@@ jarige girls in de tweede klas in de hospital zouden worden , maar hij heeft me in feite ertoe gebracht om mijn daily oef@@ en@@ hor@@ ror te zien met een nieuwe ervaring .
PRED SCORE: -59.5189

SENT 4025: #de# und ich 

PRED SCORE: -41.4567

SENT 4043: #de# wir sind also nicht nur dafür verantwortlich , die Menschen die wir lieben , vor Schick@@ sal@@ s@@ schlä@@ gen zu bewahren , sondern sie auch darauf vorzu@@ bereiten , gut damit klar zu kommen .
PRED 4043: we zijn niet alleen verantwoordelijk voor de mensen die we houden , voor de zal@@ m@@ slagen , maar ook voor de voorbere@@ iding van de goede deals .
PRED SCORE: -27.3443

SENT 4044: #de# und wir er@@ weisen unseren Kindern keinen Gef@@ allen , wenn wir ihnen das Gefühl geben , dass sie sich nicht anpassen können .
PRED 4044: we geven onze kinderen geen Gef@@ allen als we ze het gevoel geven dat ze niet kunnen aanpassen .
PRED SCORE: -12.7071

SENT 4045: #de# man muss zwei Dinge klar trennen : zum einen die medizinische Tatsache , amput@@ iert zu sein und zum anderen die gesellschaft@@ liche Meinung darüber , ob ich be@@ hindert bin oder nicht .
PRED 4045: twee dingen is duidelijk : ten eerste de medische fac@@ ul@@ teit , en de andere de social

PRED 4065: misschien kunnen we w@@ rij@@ heid zien als een verandering .
PRED SCORE: -9.7942

SENT 4066: #de# Wi@@ dr@@ igkeiten stehen für eine Veränderung , an die wir uns noch nicht angepasst haben .
PRED 4066: han@@ digheid is een verandering die we nog niet hebben aangepast .
PRED SCORE: -6.8937

SENT 4067: #de# ich glaube , den größten Schaden , den wir uns selbst zu@@ gefügt haben , ist zu glauben , dass wir normal sein sollen .
PRED 4067: ik denk dat de grootste damage die we aan onszelf hebben toe@@ gevoegd , is te geloven dat we normaal moeten zijn .
PRED SCORE: -14.2161

SENT 4068: #de# mal ehrlich %-% wer ist normal ?
PRED 4068: maar eerlijk %-% wie is normaal ?
PRED SCORE: -7.3505

SENT 4069: #de# es gibt kein Nor@@ mal .
PRED 4069: er is geen norm .
PRED SCORE: -1.7272

SENT 4070: #de# es gibt das Ü@@ bliche . das typische . aber nicht das normale .
PRED 4070: er is het gebruikelijke . de typische . maar niet de normale .
PRED SCORE: -10.2581

SENT 4071: #de# und würden S

PRED 4090: en hij noemde dit deel van zijn talk de X - factor , de mogelijkheden van het menselijk wil .
PRED SCORE: -13.1560

SENT 4091: #de# man kann einfach nicht genug betonen , wie entscheidend dieser Faktor für die Lebensqualität eines Menschen sein kann .
PRED 4091: je kunt gewoon niet genoeg benadru@@ kken hoe belangrijk dit factor is voor de quality van de wereld van een mens .
PRED SCORE: -17.9372

SENT 4092: #de# und Dr. Ke@@ an erzählte weiter , er sagte : " ich habe gelernt , dass Kinder , wenn sie nicht ständig etwas anderes gesagt bekommen , und auch wenn sie nur ein bisschen Unterstützung bekommen , wenn man Kinder sich selbst über@@ lässt , dann können Kinder sehr viel erreichen . "
PRED 4092: Dr. Ke@@ an vertelde verder , hij zei : " ik heb geleerd dat kinderen , als ze niet altijd iets anders worden gezegd , en ook als ze maar een beetje hulp krijgen als je kinderen zelf over@@ geeft , kunnen kinderen heel veel bereiken . "
PRED SCORE: -29.6353

SENT 4093: #de# sehen

PRED 4121: hij ontbreekt aan onze natuurlijke , kin@@ delijke ver@@ nieuw@@ ing en onze aan@@ geboren vermogen tot de verbeelding .
PRED SCORE: -18.0181

SENT 4122: #de# wenn wir es aber schaffen , den menschlichen Geist dabei zu unterstützen , weiterhin zu hoffen , sich selbst und andere Menschen um sich herum schön zu finden , neugierig und ph@@ ant@@ as@@ ie@@ voll zu sein dann setzen wir unsere Kräfte wirklich positiv ein .
PRED 4122: maar als we het kunnen doen om de menselijke geest te helpen , om te hopen dat we onszelf en andere mensen om zich heen mooi vinden , nieuwsgierig en fan@@ at@@ isch zijn , dan zetten we onze forces echt positief in .
PRED SCORE: -32.0776

SENT 4123: #de# wenn ein Geist über diese Qual@@ itäten verfü@@ gt , können wir eine neue Wirklichkeit und neue D@@ as@@ ein@@ sfor@@ men schaffen .
PRED 4123: als een geest deze kwal@@ iteiten heeft , kunnen we een nieuwe realiteit creëren en nieuwe D@@ as@@ ein@@ sfor@@ men .
PRED SCORE: -16.1036

SENT 4124: #de# 

In [6]:
head -n128 /opt/data/iwslt2017/tst2017/nobpe/tst2017.de-en.en

so this is James risen .
you may know him as the Pulitzer Prize - winning reporter for The New York Times .
long before anybody knew Edward Snowden 's name , risen wrote a book in which he famously exposed that the NSA was illegally wiretapping the phone calls of Americans .
but it is another chapter in that book that may have an even more lasting impact .
in it , he describes a catastrophic US intelligence operation in which the CIA quite literally handed over blueprints of a nuclear bomb to Iran .
if that sounds crazy , go read it .
it is an incredible story .
but you know who didn 't like that chapter ?
the US government .
for nearly a decade afterwards , risen was the subject of a US government investigation in which prosecutors demanded that he testify against one of his alleged sources .
and along the way , he became the face for the US government 's recent pattern of prosecuting whistleblowers and spying on journalists .
you see , under the First Amendment , the press has the ri

that sort of secrecy in America has allowed employees of school districts to rip off schoolchildren .
it has allowed scammers to rip off vulnerable investors .
this is the sort of behavior that affects all of us .
now , at Global Witness , we wanted to see what this actually looked like in practice .
how does this actually work ?
so what we did is we sent in an undercover investigator to 13 Manhattan law firms .
our investigator posed as an African minister who wanted to move suspect funds into the United States to buy a house , a yacht , a jet .
now , what was truly shocking was that all but one of those lawyers provided our investigator with suggestions on how to move those suspect funds .
these were all preliminary meetings , and none of the lawyers took us on as a client and of course no money moved hands , but it really shows the problem with the system .
it is also important to not just think about this as individual cases .
this is not just about an individual lawyer who is spok

In [7]:
outfile=/tmp/tst2017.$srclang-$tgtlang.bpe.hyp.$tgtlang
reffile=/opt/data/iwslt2017/tst2017/nobpe/tst2017.$srclang-$tgtlang.$tgtlang

sed 's/@@ //g' $outfile |
    multi-bleu.perl $reffile 2> /dev/null

BLEU = 19.27, 54.6/25.4/13.8/7.9 (BP=0.977, ratio=0.978, hyp_len=68180, ref_len=69743)


b)  Using English as pivot language, e. g. first translate to
    English, then translate to Dutch.

In [8]:
srclang=de
tgtlang=nl
pivot=en

srcfile=/opt/data/iwslt2017/tst2017/tst2017.$srclang-$tgtlang.bpe.$srclang
outfile=/tmp/tst2017.$srclang-$tgtlang.bpe.pivot

python3 NMTGMinor/translate.py \
    -gpu 0 \
    -model /opt/data/nmt_models/transformer/model.average.pt \
    -src $srcfile \
    -output $outfile \
    -beam_size 1 \
    -batch_size 256 \
    -bos_token "#${pivot}#" \
    -start_with_tag


srcfile=$outfile
outfile=/tmp/tst2017.$srclang-$tgtlang.bpe.pivothyp.$tgtlang

python3 NMTGMinor/translate.py \
    -gpu 0 \
    -model /opt/data/nmt_models/transformer/model.average.pt \
    -src $srcfile \
    -output $outfile \
    -beam_size 1 \
    -batch_size 256 \
    -bos_token "#${tgtlang}#" \
    -start_with_tag

* Starting token #en# 
['/opt/data/nmt_models/transformer/model.average.pt']
* Joining the weights of encoder and decoder word embeddings
* Joining the weights of decoder input and output embeddings
* Starting token #nl# 
['/opt/data/nmt_models/transformer/model.average.pt']
* Joining the weights of encoder and decoder word embeddings
* Joining the weights of decoder input and output embeddings


In [9]:
srclang=de
tgtlang=nl
# outfile=/tmp/tst2017.$srclang-$tgtlang.bpe.pivothyp.$tgtlang
reffile=/opt/data/iwslt2017/tst2017/nobpe/tst2017.$srclang-$tgtlang.$tgtlang

sed 's/@@ //g' $outfile |
    multi-bleu.perl $reffile 2> /dev/null

BLEU = 20.12, 57.0/27.5/15.7/9.4 (BP=0.919, ratio=0.922, hyp_len=64335, ref_len=69743)


Which one produces the better result? Could you try to explain why?

- Translation via English produces the better result
- The model is only taught to translate from/to English
- It's language independent latent representation isn't good enough